In [28]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from alpharaw import register_all_readers
from alpharaw.ms_data_base import ms_reader_provider
from dia_aspire_rescore.psm.matcher import DIAPeptideSpectrumMatcher
from dia_aspire_rescore.io import read_diann2
import logging

logging.getLogger('dia_aspire_rescore').setLevel(logging.DEBUG)
logging.getLogger('numba').setLevel(logging.WARNING) 

logging.basicConfig(
    level=logging.WARNING,  
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

register_all_readers()

In [32]:
psm_df_original = read_diann2("../../data/raw/SYS026_RA957/DDA_SYSMHC_bynam/lib-base-result-first-pass.parquet")
# decreasing rt order
psm_df_sorted = psm_df_original.sort_values(by="rt", ascending=False)
psm_df_sorted

/Users/zshang/Projects/DIA-Aspire-rescore/.venv/lib/python3.13/site-packages/alphabase/psm_reader/psm_reader.py:342: UserWarning: Unknown modifications: {'Q(UniMod:1)'}. Precursors with unknown modifications will be removed.
  warnings.warn(


,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr1_search1,fdr2_search1,fdr1_search2,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs
43522,20200317_QE_HFX2_LC3_DIA_RA957_R01,SYVGIATLI,1,119.682693,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.000000,936.540060,0.0
222413,20200317_QE_HFX2_LC3_DIA_RA957_R01,SLVGIATYI,1,119.682693,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.000000,936.540060,0.0
43933,20200317_QE_HFX2_LC3_DIA_RA957_R01,TASLFVVSL,1,119.682693,119.526230,119.932961,0.0,1/uc004ftz.1,1/uc004ftz.1,,...,0.450818,1.000000,0.0,0.0,,,9,1.000000,936.540060,0.0
228125,20200317_QE_HFX2_LC3_DIA_RA957_R01,IPFSGALEL,1,119.682693,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.000087,0.000124,0.0,0.0,,,9,1.000000,946.524410,0.0
93041,20200317_QE_HFX2_LC3_DIA_RA957_R01,IEFSGALPL,1,119.682693,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.000000,946.524410,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46342,20200317_QE_HFX2_LC3_DIA_RA957_R01,SVTSSTHEK,2,0.325708,0.043572,0.608008,0.0,1/sp|Q9NW08|RPC2_HUMAN,1/sp|Q9NW08|RPC2_HUMAN,,...,0.031545,0.001135,0.0,0.0,,,9,0.002721,488.240721,0.0
226620,20200317_QE_HFX2_LC3_DIA_RA957_R01,SETSSTHVK,2,0.325708,0.043572,0.608008,0.0,1/sp|Q9NW08|RPC2_HUMAN,1/sp|Q9NW08|RPC2_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,0.002721,488.240721,0.0
182391,20200317_QE_HFX2_LC3_DIA_RA957_R01,EAAKQMRSR,3,0.320100,0.100662,0.508281,0.0,2/sp|P36551|HEM6_HUMAN/sp|P36551-2|HEM6_HUMAN,2/sp|P36551|HEM6_HUMAN/sp|P36551-2|HEM6_HUMAN,,...,0.265428,1.000000,0.0,0.0,,,9,0.002675,359.525830,0.0
285284,20200317_QE_HFX2_LC3_DIA_RA957_R01,ESSRDTGQVK,3,0.320100,0.037964,0.602398,0.0,1/sp|A0A1B0GUI7|BRDOS_HUMAN,1/sp|A0A1B0GUI7|BRDOS_HUMAN,,...,0.492493,1.000000,0.0,0.0,,,10,0.002675,369.519398,0.0


In [33]:
spec_dict = {}
peak_dict = {}

for run in psm_df_original['raw_name'].unique():
    hdf_reader = ms_reader_provider.get_reader("hdf5")
    hdf_reader.load_raw(f"../output/{run}.mzML.hdf5")
    spec_dict[run] = hdf_reader.spectrum_df
    peak_dict[run] = hdf_reader.peak_df

spec_dict, peak_dict
    

({'20200317_QE_HFX2_LC3_DIA_RA957_R01':        isolation_lower_mz  isolation_upper_mz  ms_level  peak_start_idx  \
  0                    -1.0                -1.0         1               0   
  1                   350.0               387.0         2            3283   
  2                   386.0               416.0         2            4654   
  3                   415.0               439.0         2            5532   
  4                   438.0               462.0         2            6317   
  ...                   ...                 ...       ...             ...   
  89076               883.0               955.0         2       137080673   
  89077               954.0              1057.0         2       137082379   
  89078              1056.0              1650.0         2       137084652   
  89079                -1.0                -1.0         1       137086933   
  89080               350.0               387.0         2       137092701   
  
         peak_stop_idx  precursor_c

In [52]:
spec_dict['20200317_QE_HFX2_LC3_DIA_RA957_R01']

,isolation_lower_mz,isolation_upper_mz,ms_level,peak_start_idx,peak_stop_idx,precursor_charge,precursor_mz,rt,spec_idx
0,-1.0,-1.0,1,0,3283,0,-1.0,0.002374,0
1,350.0,387.0,2,3283,4654,0,368.5,0.006591,1
2,386.0,416.0,2,4654,5532,0,401.0,0.007747,2
3,415.0,439.0,2,5532,6317,0,427.0,0.008868,3
4,438.0,462.0,2,6317,7198,0,450.0,0.009991,4
...,...,...,...,...,...,...,...,...,...
89076,883.0,955.0,2,137080673,137082379,0,919.0,119.995220,89076
89077,954.0,1057.0,2,137082379,137084652,0,1005.5,119.996410,89077
89078,1056.0,1650.0,2,137084652,137086933,0,1353.0,119.998780,89078
89079,-1.0,-1.0,1,137086933,137092701,0,-1.0,120.000600,89079


In [ ]:
psm_df_last = psm_df_sorted.iloc[0:5]
psm_df_last.iloc[0, psm_df_last.columns.get_loc('rt')] = 1000
psm_df_last.iloc[1, psm_df_last.columns.get_loc('rt')] = 119.68
psm_df_last.iloc[2, psm_df_last.columns.get_loc('rt')] = 100
psm_df_last.iloc[3, psm_df_last.columns.get_loc('rt')] = 90
psm_df_last.iloc[4, psm_df_last.columns.get_loc('rt')] = 0.01
psm_df_last

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr1_search1,fdr2_search1,fdr1_search2,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs
43522,20200317_QE_HFX2_LC3_DIA_RA957_R01,SYVGIATLI,1,1000.00,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.0,936.54006,0.0
222413,20200317_QE_HFX2_LC3_DIA_RA957_R01,SLVGIATYI,1,119.68,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.0,936.54006,0.0
43933,20200317_QE_HFX2_LC3_DIA_RA957_R01,TASLFVVSL,1,100.00,119.526230,119.932961,0.0,1/uc004ftz.1,1/uc004ftz.1,,...,0.450818,1.000000,0.0,0.0,,,9,1.0,936.54006,0.0
228125,20200317_QE_HFX2_LC3_DIA_RA957_R01,IPFSGALEL,1,90.00,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.000087,0.000124,0.0,0.0,,,9,1.0,946.52441,0.0
93041,20200317_QE_HFX2_LC3_DIA_RA957_R01,IEFSGALPL,1,0.01,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,1.000000,1.000000,0.0,0.0,,,9,1.0,946.52441,0.0


In [35]:

matcher = DIAPeptideSpectrumMatcher()
matcher.max_spec_per_query = 3
psm_df, fragment_mz_df, matched_intensity_df, matched_mz_err_df = matcher.match_ms2_multi_raw(
    psm_df_last,
    {'20200317_QE_HFX2_LC3_DIA_RA957_R01': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5'},
    'hdf5',
)
    

  0%|          | 0/1 [00:00<?, ?it/s]2025-11-04 14:54:51,411 - dia_aspire_rescore.psm.matcher - DEBUG - Processing DIA group: -1
2025-11-04 14:54:51,412 - dia_aspire_rescore.psm.matcher - DEBUG - PSM idxes in this DIA group: []
2025-11-04 14:54:51,412 - dia_aspire_rescore.psm.matcher - DEBUG - Processing DIA group: 368
2025-11-04 14:54:51,412 - dia_aspire_rescore.psm.matcher - DEBUG - PSM idxes in this DIA group: []
2025-11-04 14:54:51,412 - dia_aspire_rescore.psm.matcher - DEBUG - Processing DIA group: 401
2025-11-04 14:54:51,413 - dia_aspire_rescore.psm.matcher - DEBUG - PSM idxes in this DIA group: []
2025-11-04 14:54:51,413 - dia_aspire_rescore.psm.matcher - DEBUG - Processing DIA group: 427
2025-11-04 14:54:51,413 - dia_aspire_rescore.psm.matcher - DEBUG - PSM idxes in this DIA group: []
2025-11-04 14:54:51,413 - dia_aspire_rescore.psm.matcher - DEBUG - Processing DIA group: 450
2025-11-04 14:54:51,413 - dia_aspire_rescore.psm.matcher - DEBUG - PSM idxes in this DIA group: []
2025

In [36]:
psm_df

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs,frag_start_idx,frag_stop_idx,spec_idx
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,SYVGIATLI,1,1000.00,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,0.0,,,9,1.0,936.54006,0.0,0,8,89030
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,SLVGIATYI,1,119.68,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,0.0,,,9,1.0,936.54006,0.0,8,16,88823
2,20200317_QE_HFX2_LC3_DIA_RA957_R01,TASLFVVSL,1,100.00,119.526230,119.932961,0.0,1/uc004ftz.1,1/uc004ftz.1,,...,0.0,,,9,1.0,936.54006,0.0,16,24,74195
3,20200317_QE_HFX2_LC3_DIA_RA957_R01,IPFSGALEL,1,90.00,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.0,,,9,1.0,946.52441,0.0,24,32,66743
4,20200317_QE_HFX2_LC3_DIA_RA957_R01,IEFSGALPL,1,0.01,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.0,,,9,1.0,946.52441,0.0,32,40,20
5,20200317_QE_HFX2_LC3_DIA_RA957_R01,SYVGIATLI,1,1000.00,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,0.0,,,9,1.0,936.54006,0.0,40,48,89053
6,20200317_QE_HFX2_LC3_DIA_RA957_R01,SLVGIATYI,1,119.68,119.557571,119.870628,0.0,1/sp|Q7Z6Z7|HUWE1_HUMAN,1/sp|Q7Z6Z7|HUWE1_HUMAN,,...,0.0,,,9,1.0,936.54006,0.0,48,56,88846
7,20200317_QE_HFX2_LC3_DIA_RA957_R01,TASLFVVSL,1,100.00,119.526230,119.932961,0.0,1/uc004ftz.1,1/uc004ftz.1,,...,0.0,,,9,1.0,936.54006,0.0,56,64,74218
8,20200317_QE_HFX2_LC3_DIA_RA957_R01,IPFSGALEL,1,90.00,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.0,,,9,1.0,946.52441,0.0,64,72,66766
9,20200317_QE_HFX2_LC3_DIA_RA957_R01,IEFSGALPL,1,0.01,119.401268,119.870628,0.0,1/sp|Q9NTK5|OLA1_HUMAN,1/sp|Q9NTK5|OLA1_HUMAN,,...,0.0,,,9,1.0,946.52441,0.0,72,80,43


In [48]:
# real world scenairo with multiple raw files
matcher_multi_raw = DIAPeptideSpectrumMatcher()
matcher_multi_raw.max_spec_per_query = 5
psm_df, fragment_mz_df, matched_intensity_df, matched_mz_err_df = matcher_multi_raw.match_ms2_multi_raw(
    psm_df_original,
    {'20200317_QE_HFX2_LC3_DIA_RA957_R01': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R01.mzML.hdf5',
     '20200317_QE_HFX2_LC3_DIA_RA957_R02': '../output/20200317_QE_HFX2_LC3_DIA_RA957_R02.mzML.hdf5'},
    'hdf5',
)
    

100%|██████████| 2/2 [01:10<00:00, 35.02s/it]


In [49]:
psm_df

,raw_name,sequence,charge,rt,rt_start,rt_stop,mobility,proteins,uniprot_ids,genes,...,fdr2_search2,mods,mod_sites,nAA,rt_norm,precursor_mz,ccs,frag_start_idx,frag_stop_idx,spec_idx
0,20200317_QE_HFX2_LC3_DIA_RA957_R01,YQYYHRYY,3,44.166344,44.104832,44.227886,0.0,1/sp|O96000|NDUBA_HUMAN,1/sp|O96000|NDUBA_HUMAN,,...,0.0,,,8,0.369029,419.189212,0.0,0,7,32479
1,20200317_QE_HFX2_LC3_DIA_RA957_R01,FRPDNFVF,2,95.625778,95.502647,95.717796,0.0,1/sp|Q13885|TBB2A_HUMAN,1/sp|Q13885|TBB2A_HUMAN,,...,0.0,,,8,0.798994,521.261259,0.0,14,21,70916
2,20200317_QE_HFX2_LC3_DIA_RA957_R01,RLDSPVMV,2,51.800659,51.708340,51.893513,0.0,1/sp|P60510|PP4C_HUMAN,1/sp|P60510|PP4C_HUMAN,,...,0.0,,,8,0.432817,458.749670,0.0,28,35,38138
3,20200317_QE_HFX2_LC3_DIA_RA957_R01,YTAAVPYR,2,50.818192,50.725948,50.879574,0.0,6/tr|E9PM00|E9PM00_HUMAN/tr|B4E1K5|B4E1K5_HUMA...,6/tr|E9PM00|E9PM00_HUMAN/tr|B4E1K5|B4E1K5_HUMA...,,...,0.0,,,8,0.424608,470.747985,0.0,35,42,37426
4,20200317_QE_HFX2_LC3_DIA_RA957_R01,NYMDAQYY,2,106.851646,106.789871,106.974739,0.0,1/sp|P07339|CATD_HUMAN,1/sp|P07339|CATD_HUMAN,,...,0.0,,,8,0.892791,534.210575,0.0,49,56,79266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694190,20200317_QE_HFX2_LC3_DIA_RA957_R02,TRIDAHNGVAPSSR,3,27.969645,27.906984,28.094990,0.0,1/sp|Q99986|VRK1_HUMAN,1/sp|Q99986|VRK1_HUMAN,,...,0.0,,,14,0.233874,494.258689,0.0,14238411,14238424,20568
1694191,20200317_QE_HFX2_LC3_DIA_RA957_R02,ETIQDNHEDGPAFK,3,42.191280,42.034897,42.284889,0.0,1/sp|P13010|XRCC5_HUMAN,1/sp|P13010|XRCC5_HUMAN,,...,0.0,,,14,0.352791,534.246115,0.0,14238424,14238437,31058
1694192,20200317_QE_HFX2_LC3_DIA_RA957_R02,NLVGQLAEGGYPTR,2,74.645042,74.553070,74.767960,0.0,1/sp|Q9Y2H8|ZN510_HUMAN,1/sp|Q9Y2H8|ZN510_HUMAN,,...,0.0,,,14,0.624159,737.886072,0.0,14238450,14238463,55193
1694193,20200317_QE_HFX2_LC3_DIA_RA957_R02,EVYEEAGVKGKLGR,3,41.783123,41.657825,41.971107,0.0,7/sp|hsa_circ_0008916_0|hsa_circ_0008916_0/sp|...,7/sp|hsa_circ_0008916_0|hsa_circ_0008916_0/sp|...,,...,0.0,,,14,0.349378,512.278979,0.0,14238476,14238489,30781
